# Notebook de Treinamento e Anotações: Curso de Classificação com Machine Learning

Este notebook serviu como registro prático e teórico no meu aprendizado de Machine Learning.

`Enriqueci este notebook com anotações adicionais e aplicações práticas tornando-o uma referência valiosa para consultas e implementações em futuros projetos reais.`

Espero que este material inspire outros a explorar ainda mais o fascinante mundo do Machine Learning. 

1. **Análise Exploratória de Dados**  
   - Exploração inicial para compreender as características e padrões presentes na base de dados.

2. **Preparação dos Dados**  
   - Aplicação de transformações essenciais para otimizar o desempenho dos modelos:
     - **Normalização de Dados Numéricos**: Ajuste dos valores numéricos para uma escala comum.
     - **Dummificação de Dados Categóricos Não Binários**: Conversão de variáveis categóricas em variáveis dummy (variáveis indicadoras).
     - **Binarização de Dados Categóricos Binários**: Transformação de variáveis categóricas binárias em formato adequado para o modelo.

3. **Divisão dos Dados em Conjuntos de Treino e Teste**  
   - Métodos para dividir os dados:
     - **Divisão Simples**: Separação direta dos dados em treino e teste.
     - **Divisão Estratificada**: Separação dos dados levando em conta as proporções das classes da variavel target.

4. **Treinamento do Modelo de Classificação**  
   - Construção e ajuste de modelos para prever as classes dos dados.

5. **Avaliação do Modelo**  
   - Utilização de métricas de desempenho e da Curva ROC (AUC) para avaliar a eficácia do modelo.

Compartilhar conhecimento é uma alegria—viva ao aprendizado contínuo, boa pratica e bons estudo a quem estiver lendo, abraços!

# Carregamento dos Dados

In [13]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

#ignorando Warning inuteis
import warnings 
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

pd.set_option('display.max_info_columns', 500)
pd.set_option('display.max_info_rows', 500)

In [14]:
#CRIANDO DADOS FICTICIOS
np.random.seed(1)

Tamanho_da_amostra = 500
n = Tamanho_da_amostra/2 ; n = int(n)
df = pd.DataFrame()

df['desfecho'] = ['não'] * n + ['sim'] * n
filtro_1 = df['desfecho'] == 'não'
filtro_0 = df['desfecho'] == 'sim'

lista_col = ['idade','sexo','tabagismo','avc_previo','hipo','has',
             'dm','doenca_cardiaca','var_num1','var_num2']
for coluna in lista_col: df[coluna] = np.nan

df['sexo'][filtro_0] = np.random.choice(['F', 'M'], size=n, p=[0.4, 0.6])
df['sexo'][filtro_1] = np.random.choice(['F', 'M'], size=n, p=[0.7, 0.3])

df['idade'][filtro_0] = np.random.normal(loc=30, scale=5, size=n).round().astype(int)
df['idade'][filtro_1] = np.random.normal(loc=23, scale=4, size=n).round().astype(int)

df['tabagismo'][filtro_0] = np.random.choice(['não', 'sim'], size=n, p=[0.3, 0.7])
df['tabagismo'][filtro_1] = np.random.choice(['não', 'sim'], size=n, p=[0.4, 0.6])

df['avc_previo'][filtro_0] = np.random.choice(['não', 'sim'], size=n, p=[0.3, 0.7])
df['avc_previo'][filtro_1] = np.random.choice(['não', 'sim'], size=n, p=[0.5, 0.5])

df['hipo'][filtro_0] = np.random.choice(['não', 'sim'], size=n, p=[0.2, 0.8])
df['hipo'][filtro_1] = np.random.choice(['não', 'sim'], size=n, p=[0.3, 0.7])

df['has'][filtro_0] = np.random.choice(['não', 'sim'], size=n, p=[0.5, 0.5])
df['has'][filtro_1] = np.random.choice(['não', 'sim'], size=n, p=[0.2, 0.8])

df['dm'][filtro_0] = np.random.choice(['não', 'sim'], size=n, p=[0.8, 0.2])
df['dm'][filtro_1] = np.random.choice(['não', 'sim'], size=n, p=[0.3, 0.7])

df['doenca_cardiaca'][filtro_0] = np.random.choice(['não', 'sim'], size=n, p=[0.2, 0.8])
df['doenca_cardiaca'][filtro_1] = np.random.choice(['não', 'sim'], size=n, p=[0.6, 0.4])

df['var_num1'][filtro_0] = np.random.poisson(12,n)
df['var_num1'][filtro_1] = np.random.poisson(20,n)

df['var_num2'][filtro_0] = np.random.uniform(0,100,n)
df['var_num2'][filtro_1] = np.random.uniform(0,100,n)

df.head()

,desfecho,idade,sexo,tabagismo,avc_previo,hipo,has,dm,doenca_cardiaca,var_num1,var_num2
0,não,21.0,F,não,não,sim,sim,não,sim,22.0,78.568726
1,não,25.0,M,não,sim,sim,sim,sim,sim,28.0,21.299272
2,não,19.0,F,sim,não,sim,sim,sim,não,23.0,66.262434
3,não,21.0,M,sim,sim,sim,sim,sim,não,23.0,52.336710
4,não,27.0,F,sim,sim,não,sim,não,não,17.0,17.829723


In [15]:
#CRIANDO DADOS FICTICIOS 2
from sklearn.datasets import make_classification
np.random.seed(3141592)

# Gerando dados numéricos e a variável dependente 'y'
x, y = make_classification(n_samples=500, 
                           n_features=10, #total de colunas
                           n_informative=5, #colunas informativas
                           n_redundant=5, #colunas redundantes
                           n_classes=2, #numero de classes y
                           random_state=3141592)

df = pd.DataFrame(x)
df.columns = ['x_num'+str(i) for i in range(10)]

df['y'] = y.astype('O')

# Adicionando 10 variáveis categóricas binarias

for i in range(5):
    if i >= 3:
        df['x_bin'+str(i)] = np.random.choice(['cat0','cat1'], size=len(df))
    else:
        df['x_bin'+str(i)] = np.nan
        df['x_bin'+str(i)][df['y'] == 0] = np.random.choice(['cat0','cat1'], size=len(df), p=[.7,.3])
        df['x_bin'+str(i)][df['y'] == 1] = np.random.choice(['cat0','cat1'], size=len(df), p=[.3,.7])

for i in range(3):
    if i >= 2:
        df['x_cat'+str(i)] = np.random.choice(['cat0','cat1','cat2',], size=len(df))
    else:
        df['x_cat'+str(i)] = np.nan
        df['x_cat'+str(i)][df['y'] == 0] = np.random.choice(['cat0','cat1','cat2',], size=len(df), p=[.3,.3,.4])
        df['x_cat'+str(i)][df['y'] == 1] = np.random.choice(['cat0','cat1','cat2',], size=len(df), p=[.2,.7,.1])

for i in range(2):
    if i >= 1:
        df['x_cat_'+str(i)] = np.random.choice(['cat0','cat1','cat2','cat3'], size=len(df))
    else:
        df['x_cat_'+str(i)] = np.nan
        df['x_cat_'+str(i)][df['y'] == 0] = np.random.choice(['cat0','cat1','cat2','cat3'], size=len(df), p=[.4,.3,.2,.1])
        df['x_cat_'+str(i)][df['y'] == 1] = np.random.choice(['cat0','cat1','cat2','cat3'], size=len(df), p=[.1,.2,.3,.4])

df.head()

,x_num0,x_num1,x_num2,x_num3,x_num4,x_num5,x_num6,x_num7,x_num8,x_num9,...,x_bin0,x_bin1,x_bin2,x_bin3,x_bin4,x_cat0,x_cat1,x_cat2,x_cat_0,x_cat_1
0,0.843910,1.012945,1.190556,2.672900,-0.943966,-2.093581,-0.674588,0.598498,-1.244136,-0.560193,...,cat0,cat0,cat1,cat1,cat1,cat0,cat2,cat0,cat0,cat0
1,-1.271782,1.820219,1.225230,1.055635,-0.365865,-2.013976,1.250390,1.054216,-0.362961,0.381651,...,cat0,cat1,cat0,cat0,cat0,cat2,cat1,cat1,cat0,cat2
2,0.097646,2.428925,2.313740,3.347566,-2.453440,-3.909484,-3.100085,-0.070826,-3.842795,-0.606800,...,cat0,cat1,cat0,cat1,cat1,cat1,cat2,cat2,cat2,cat0
3,-1.680626,-1.532230,1.490382,1.053494,-0.385167,1.225927,0.676873,1.031453,-1.969521,1.538824,...,cat0,cat1,cat0,cat0,cat1,cat1,cat2,cat0,cat3,cat3
4,-0.187534,1.967675,2.223379,1.808796,-2.786070,-0.466622,-0.334265,-0.897495,-2.092557,0.052721,...,cat0,cat0,cat1,cat0,cat1,cat2,cat1,cat1,cat3,cat1


# Analises Iniciais

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 21 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   x_num0   500 non-null    float64
 1   x_num1   500 non-null    float64
 2   x_num2   500 non-null    float64
 3   x_num3   500 non-null    float64
 4   x_num4   500 non-null    float64
 5   x_num5   500 non-null    float64
 6   x_num6   500 non-null    float64
 7   x_num7   500 non-null    float64
 8   x_num8   500 non-null    float64
 9   x_num9   500 non-null    float64
 10  y        500 non-null    object 
 11  x_bin0   500 non-null    object 
 12  x_bin1   500 non-null    object 
 13  x_bin2   500 non-null    object 
 14  x_bin3   500 non-null    object 
 15  x_bin4   500 non-null    object 
 16  x_cat0   500 non-null    object 
 17  x_cat1   500 non-null    object 
 18  x_cat2   500 non-null    object 
 19  x_cat_0  500 non-null    object 
 20  x_cat_1  500 non-null    object 
dtypes: float64(10), 

In [17]:
display(df.describe(include='all'))

,x_num0,x_num1,x_num2,x_num3,x_num4,x_num5,x_num6,x_num7,x_num8,x_num9,...,x_bin0,x_bin1,x_bin2,x_bin3,x_bin4,x_cat0,x_cat1,x_cat2,x_cat_0,x_cat_1
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,...,500,500,500,500,500,500,500,500,500,500
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,2,2,2,2,3,3,3,4,4
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,cat1,cat0,cat0,cat1,cat1,cat1,cat1,cat0,cat0,cat1
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,261,252,251,260,269,253,267,169,135,133
mean,-0.557939,0.496546,0.352891,0.476149,0.204914,-1.040651,0.454461,0.730617,-0.109831,0.177424,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,1.563045,1.604209,1.330140,1.478911,1.674349,1.185159,1.461027,1.365781,2.134687,0.954316,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,-4.844953,-4.911777,-3.387502,-4.537776,-5.118946,-5.115548,-5.506776,-3.930646,-5.804851,-2.896970,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,-1.627864,-0.488564,-0.531668,-0.520949,-0.847893,-1.798338,-0.437514,-0.225202,-1.761611,-0.538396,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,-0.629553,0.701758,0.337476,0.494579,0.215876,-1.096979,0.678468,0.624379,-0.303252,0.247174,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,0.403434,1.551373,1.244882,1.561551,1.366376,-0.271156,1.405160,1.650740,1.407978,0.853892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
import plotly.express as px

import scipy.stats as stats
from scipy.stats import chi2_contingency

In [19]:
var_dep = 'y'
px.histogram(df, x=var_dep, text_auto=True, color=var_dep)

for coluna in df.columns:
    if df[coluna].dtype == 'O':
        print('='*15, coluna, '='*15)
        fig = px.histogram(df, x=coluna, text_auto=True, color=coluna)
        fig.show()
    elif df[coluna].dtype == 'int64' or df[coluna].dtype == 'float64':
        fig = px.box(df, x=coluna)
        fig.show()

In [21]:
coluna_p_menor_05 = []

for coluna in df.columns:
    if df[coluna].dtype == 'O':
        
        tabela_contingencia = pd.crosstab(df[var_dep], df[coluna])
        chi2, p, dof, expected = chi2_contingency(tabela_contingencia)
        print('P-value:', round(p,4))
        
        fig = px.histogram(df, x=coluna, text_auto=True, color=var_dep, barmode='group')
        fig.show()

        if p < 0.10:
            coluna_p_menor_05.append(coluna)

P-value: 0.0


P-value: 0.0


P-value: 0.0


P-value: 0.0


P-value: 0.9971


...

P-value: 0.0


P-value: 0.0


P-value: 0.7328


P-value: 0.0


P-value: 0.3515


In [1]:
for coluna in df.columns:
    if df[coluna].dtype == 'int64' or df[coluna].dtype == 'float64':
        grupos = df.groupby(var_dep)[coluna].apply(list)
        
        if len(df[var_dep].unique()) == 2:
            # Extração dos grupos para aplicação do teste de Mann-Whitney
            grupo1, grupo2 = grupos
            stat, p_value_mw = stats.mannwhitneyu(grupo1, grupo2)
            print('P-value (Mann-Whitney):', round(p_value_mw, 4))

            # Aplicação do teste T de Student se os dados parecem seguir uma distribuição normal
            stat, p_value_t = stats.ttest_ind(grupo1, grupo2)
            print('P-value (Teste T):', round(p_value_t, 4))

            if p_value_mw < 0.05 or p_value_t < 0.05:
                coluna_p_menor_05.append(coluna)

        elif len(df[var_dep].unique()) > 2:
            # Aplicando o teste ANOVA
            f_value, p_value_kk = stats.f_oneway(*grupos)
            print('P-value (Kruskal-Wallis):', round(p_value_kk, 4))
            
            # Aplicando o teste de Kruskal-Wallis
            kruskal_stat, p_value_anova = stats.kruskal(*grupos)
            print('P-value (Anova):', round(p_value_anova, 4))
            
            if p_value_kk < 0.05 or p_value_anova < 0.05:
                coluna_p_menor_05.append(coluna)
    
        fig = px.violin(df, x=coluna, y=var_dep, color=var_dep, box=True, points="all", hover_data=df.columns)
        #px.box(df, x=coluna, y=var_dep, color=var_dep)
        fig.show()

NameError: name 'df' is not defined

In [23]:
coluna_p_menor_05

['y',
 'x_bin0',
 'x_bin1',
 'x_bin2',
 'x_cat0',
 'x_cat1',
 'x_cat_0',
 'x_num0',
 'x_num1',
 'x_num2',
 'x_num3',
 'x_num4',
 'x_num6',
 'x_num7',
 'x_num8',
 'x_num9']

In [24]:
if var_dep in coluna_p_menor_05:
    coluna_p_menor_05.remove(var_dep)

In [25]:
x = df[coluna_p_menor_05]
y = df[var_dep]

# Transformando Variaveis

## Independentes: Dummys

In [26]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder #transformando colunas com 2 categorias em 0 e 1

In [27]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   x_bin0   500 non-null    object 
 1   x_bin1   500 non-null    object 
 2   x_bin2   500 non-null    object 
 3   x_cat0   500 non-null    object 
 4   x_cat1   500 non-null    object 
 5   x_cat_0  500 non-null    object 
 6   x_num0   500 non-null    float64
 7   x_num1   500 non-null    float64
 8   x_num2   500 non-null    float64
 9   x_num3   500 non-null    float64
 10  x_num4   500 non-null    float64
 11  x_num6   500 non-null    float64
 12  x_num7   500 non-null    float64
 13  x_num8   500 non-null    float64
 14  x_num9   500 non-null    float64
dtypes: float64(9), object(6)
memory usage: 58.7+ KB


In [28]:
colunas_binarias = []
for coluna in x.columns:
    if df[coluna].dtype == 'O':
        categorias = x[coluna].unique()
        if len(categorias) == 2:
            print('2 niveis:', coluna, '=>', categorias)
            colunas_binarias.append(coluna)
        else:
            print('3 niveis:', coluna, '=>', categorias)
            colunas_binarias.append(coluna)

2 niveis: x_bin0 => ['cat0' 'cat1']
2 niveis: x_bin1 => ['cat0' 'cat1']
2 niveis: x_bin2 => ['cat1' 'cat0']
3 niveis: x_cat0 => ['cat0' 'cat2' 'cat1']
3 niveis: x_cat1 => ['cat2' 'cat1' 'cat0']
3 niveis: x_cat_0 => ['cat0' 'cat2' 'cat3' 'cat1']


In [29]:
coluna = x.columns
one_hot = make_column_transformer((
    OneHotEncoder(drop='if_binary'), #caso a coluna tenha apenas 2 categorias 
    colunas_binarias), #passando quais são essas colunas
                                  remainder = 'passthrough', sparse_threshold=0) #oque deve ser feito com as outras

#Aplicando transformação
x = one_hot.fit_transform(x)

#Os novos nomes das colunas #'onehotencoder=transformadas; 'remainder'=não transformadas
novos_nomes_colunas = one_hot.get_feature_names_out(coluna)

x = pd.DataFrame(x, columns = novos_nomes_colunas) #alterando de volta
x_columns = x.columns.tolist() 
x.head()

,onehotencoder__x_bin0_cat1,onehotencoder__x_bin1_cat1,onehotencoder__x_bin2_cat1,onehotencoder__x_cat0_cat0,onehotencoder__x_cat0_cat1,onehotencoder__x_cat0_cat2,onehotencoder__x_cat1_cat0,onehotencoder__x_cat1_cat1,onehotencoder__x_cat1_cat2,onehotencoder__x_cat_0_cat0,...,onehotencoder__x_cat_0_cat3,remainder__x_num0,remainder__x_num1,remainder__x_num2,remainder__x_num3,remainder__x_num4,remainder__x_num6,remainder__x_num7,remainder__x_num8,remainder__x_num9
0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.843910,1.012945,1.190556,2.672900,-0.943966,-0.674588,0.598498,-1.244136,-0.560193
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,-1.271782,1.820219,1.225230,1.055635,-0.365865,1.250390,1.054216,-0.362961,0.381651
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.097646,2.428925,2.313740,3.347566,-2.453440,-3.100085,-0.070826,-3.842795,-0.606800
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,-1.680626,-1.532230,1.490382,1.053494,-0.385167,0.676873,1.031453,-1.969521,1.538824
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,-0.187534,1.967675,2.223379,1.808796,-2.786070,-0.334265,-0.897495,-2.092557,0.052721


### Normalização de dados

In [30]:
from sklearn.preprocessing import MinMaxScaler
normalizacao = MinMaxScaler()
x = normalizacao.fit_transform(x)

#df['Close_padronizada'] = (df[coluna] - df[coluna].mean()) / df[coluna].std()
#df['Close_normalizada'] = (df[coluna] - df[coluna].min()) / (df[coluna].max() - df[coluna].min())

## Dependente

In [31]:
from sklearn.preprocessing import LabelEncoder
y = LabelEncoder().fit_transform(y)

# Dividindo a Base em Treino e Teste 

In [32]:
from sklearn.model_selection import train_test_split
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, 
                                                    stratify = y, #para manter a proporção da Var Dep nos splits
                                                    random_state = 5) #raiz da aleatoridade
# test_size = 0.25 #porcentagem que ira ser separado para testes

x_treino.shape, x_teste.shape
y_treino.shape, y_teste.shape

((375,), (125,))

# Códigos de Avaliação do Modelo

In [33]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, RocCurveDisplay


def avaliar_modelo(y_verdadeiro, y_predito, conjunto_nome):
    """
    Função para avaliar e imprimir o relatório de classificação e a matriz de confusão.
    """
    #if conjunto_nome == "Treino":
     #   pass
    #else:
    print('*' * 70)
    print("Relatório de Classificação para o Conjunto de", conjunto_nome,":\n")
    print(classification_report(y_verdadeiro, y_predito))
    
    print('*' * 55)
    
    print("Matriz de Confusão para o Conjunto de", conjunto_nome,":\n")
    print(confusion_matrix(y_verdadeiro, y_predito))
    print('*' * 70)

    display(RocCurveDisplay.from_predictions(y_verdadeiro, y_predito, name = conjunto_nome))